In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv()

True

In [2]:
model = ChatOpenAI(model='gpt-4o-mini')

In [3]:
class SentimentSchema(BaseModel):
    sentiment: Literal["positive","negative"] = Field(description="Sentiment of the review")
    

In [4]:
structured_mdel = model.with_structured_output(SentimentSchema)

In [6]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal["positive","negative"]
    diagnosis: dict
    response: str

In [23]:
def find_sentiment(state: ReviewState):
    prompt = f'For the following review find out the sentiment \n - {state['review']}'
    sentiment = structured_mdel.invoke(prompt).sentiment
    return {'sentiment':sentiment}

In [44]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description="The category of the issue mentioned in the review")
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description="he emotional tone expressed by the user")
    urgency: Literal["low","medium","high"] = Field(description="How urgent or critical the issue appears to be")

diag_model = model.with_structured_output(DiagnosisSchema)

def run_diagnosis(state: ReviewState):
    prompt = f"Diagnose this negative review:\n\n{state['review']}\n return issue_type, tone, urgency"
    response = diag_model.invoke(prompt)
    return {'diagnosis': response.model_dump()}

In [35]:
prompt = f"Diagnose this negative review:\n\n{review2}\n return issue_type, tone, urgency"
response = diag_model.invoke(prompt)

In [37]:
response.model_dump()

{'issue_type': 'Bug', 'tone': 'frustrated', 'urgency': 'high'}

In [38]:
def negative_response(state: ReviewState):
    diagnosis = state['diagnosis']
    prompt = f"""You are a support assistant.
    The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'
    Write a empathetic, helpful resolution message.
    """
    response = model.invoke(prompt).content
    return {'response': response}

In [39]:
def positive_response(state: ReviewState):
    prompt = f"Write a warm than you message in response to this review:\n\n{state['review']}"
    response = model.invoke(prompt).content
    return {'response': response}

In [40]:
def sentiment_based_navigation(state:ReviewState) -> Literal["positive_response","run_diagnosis"]:
    if state['sentiment'] == 'positive':
        return 'positive_response'
    else:
        return 'run_diagnosis'

In [46]:
# define a graph
graph = StateGraph(ReviewState)

# add nodes
graph.add_node('find_sentiment', find_sentiment)
graph.add_node('run_diagnosis', run_diagnosis)
graph.add_node('negative_response', negative_response)
graph.add_node('positive_response', positive_response)

# add edges
graph.add_edge(START,'find_sentiment')
graph.add_conditional_edges('find_sentiment', sentiment_based_navigation)
graph.add_edge('run_diagnosis','negative_response')
graph.add_edge('negative_response', END)
graph.add_edge('positive_response',END)

# compile
workflow = graph.compile()

In [30]:
review1 = """I've been using this app for about a month now, and I must say, the user interface is incredibly clean and intutive.
Everything is exactly where you'd expect it to be. It's rare to find something that just works without needing a tutorial.
Great job to the design team!"""

In [31]:
initial_state = {'review': review1}
final_state = workflow.invoke(initial_state)

In [32]:
final_state

{'review': "I've been using this app for about a month now, and I must say, the user interface is incredibly clean and intutive.\nEverything is exactly where you'd expect it to be. It's rare to find something that just works without needing a tutorial.\nGreat job to the design team!",
 'sentiment': 'positive',
 'response': "Dear [Reviewer’s Name],\n\nThank you so much for your wonderful review! We're thrilled to hear that you've found our app to be clean and intuitive. It’s our goal to create an experience that feels seamless and user-friendly, so your feedback truly means a lot to us.\n\nWe'll be sure to pass along your kind words to our design team—they will be delighted to know their hard work has made a positive impact. If you have any more thoughts or suggestions as you continue to use the app, please don’t hesitate to reach out.\n\nThanks again for being a part of our community!\n\nWarm regards,  \n[Your Name]  \n[Your Position]  \n[Your Company]  "}

In [42]:
review2 = """I've been trying to login for over an hour now, and the app keeps freezing on the authentication screen. 
I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality.
"""

In [47]:
initial_state = {'review': review2}
final_state = workflow.invoke(initial_state)

final_state

{'review': "I've been trying to login for over an hour now, and the app keeps freezing on the authentication screen. \nI even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality.\n",
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'Bug', 'tone': 'frustrated', 'urgency': 'high'},
 'response': "Subject: We're Here to Help with Your Issue\n\nHi [User's Name],\n\nI truly understand how frustrating it can be to encounter bugs, especially when you’re trying to accomplish something important. I want to assure you that we’re here to help you resolve this as quickly as possible.\n\nCould you please provide me with a few more details about the issue you’re experiencing? Specifically, any error messages you’re seeing or the steps leading up to the problem would be incredibly helpful.\n\nThank you for your patience as we work through this together. Your satisfaction is our priority, and I’m committed to getting this resolved f